In [7]:
from pymongo import MongoClient
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

from utils import *

client = MongoClient('localhost', 27017)
db = client['diva-proj']
papers = db.papers

In [9]:
query = {
    '$and': [{"concepts.id": "https://openalex.org/C75684735"}, {"concepts.id": "https://openalex.org/C175801342"}]
}

In [10]:
asd = list(papers.find(query))

MemoryError: 

In [ ]:
asd

In [11]:
year_windows = list(iterate_windows_over_range(2015, 2022))
start, end = year_windows[0]
papers = get_papers_in_window(db.papers, start, end)
# papers = [p for p in papers if p['cited_by_count'] > 50]
ml_concepts = set(c['id'] for c in db.concepts.find() if c['level'] == 1)

concept_paper_map = defaultdict(float)
concept_num_map = defaultdict(int)
for p in papers:
    for c in p['concepts']:
        if c['id'] in ml_concepts:
            concept_paper_map[(c['id'], c['display_name'])] += float(c['score'])
            concept_num_map[(c['id'], c['display_name'])] += 1

g = nx.Graph()
g.add_nodes_from([name for _, name in concept_paper_map.keys()])
for k,v in concept_paper_map.items():
    _, name = k
    g.nodes[name]['points'] = v / concept_num_map[k]


edges = []
for p in papers:
        concepts = (c for c in p['concepts'] if c['id'] in ml_concepts)
        # concepts = (c for c in concepts if float(c['score']) > 0.4)
        concepts = (c['display_name'] for c in concepts)
        edges.extend(list(itertools.combinations(concepts, 2)))

edges = [sorted(e) for e in edges]
edges = [(e[0], e[1]) for e in edges]

# g.add_edges_from(edges)
# for p,q in g.edges:
#     w = (g.nodes[p]['points'] + g.nodes[q]['points']) / 2
#     g.edges[p,q]['weight'] = w

In [12]:
counter = Counter(edges)

In [14]:
counter.items()

dict_items([(('Algorithm', 'Artificial intelligence'), 2323), (('Artificial intelligence', 'Data mining'), 4726), (('Artificial intelligence', 'Machine learning'), 16775), (('Algorithm', 'Data mining'), 751), (('Algorithm', 'Machine learning'), 2404), (('Data mining', 'Machine learning'), 4950), (('Artificial intelligence', 'Natural language processing'), 1084), (('Artificial intelligence', 'Programming language'), 26), (('Machine learning', 'Natural language processing'), 1084), (('Natural language processing', 'Programming language'), 5), (('Machine learning', 'Programming language'), 26), (('Artificial intelligence', 'Human–computer interaction'), 527), (('Human–computer interaction', 'Machine learning'), 538), (('Artificial intelligence', 'Information retrieval'), 865), (('Information retrieval', 'Machine learning'), 895), (('Data mining', 'Data science'), 411), (('Data science', 'Machine learning'), 609), (('Artificial intelligence', 'Data science'), 531), (('Data mining', 'Inform

In [10]:
edges

[]